In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_dirs

In [2]:
%pwd

'd:\\My DL Workstation\\Krish\\Complete End to End Deep Learning Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\My DL Workstation\\Krish\\Complete End to End Deep Learning Project'

In [8]:
# Update entity
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

In [9]:
# Update configuration manager
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_dirs([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        data_ingestion = self.config.data_ingestion
        prepare_base_model = self.config.prepare_base_model
        training = self.config.training
        params = self.params
        training_data = os.path.join(data_ingestion.unzip_dir, "kidney-ct-scan-image")
        
        create_dirs([training.root_dir])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        
        return training_config

In [10]:
# Update component
class Training:
    def __init__(self, config: TrainingConfig, params_filepath = PARAMS_FILE_PATH):
        self.config = config
        self.params = read_yaml(params_filepath)
        
    # loads model from directory    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
    # generates the train_val split    
    def train_val_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= 'validation',
            shuffle=False,
            **dataflow_kwargs
        )
        
        # If augmentation = True, augment the training images using the augmentation parameters below
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = self.params.ROTATION_RANGE,
                horizontal_flip = self.params.HORIZONTAL_FLIP,
                width_shift_range = self.params.WIDTH_SHIFT_RANGE,
                height_shift_range = self.params.HEIGHT_SHIFT_RANGE,
                shear_range = self.params.SHEAR_RANGE,
                zoom_range = self.params.ZOOM_RANGE,
                **datagenerator_kwargs
            )
            
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            
        )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
            )

In [11]:
# Update pipeline
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_val_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-11-17 02:31:05,820: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-17 02:31:05,822: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-17 02:31:05,823: INFO: common: created directory: artifacts]
[2023-11-17 02:31:05,823: INFO: common: created directory: artifacts/training]
[2023-11-17 02:31:05,825: INFO: common: yaml file: params.yaml loaded successfully]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/200
23/23 [==============================] - 13s 579ms/step - loss: 10.9963 - accuracy: 0.5927 - val_loss: 23.5386 - val_accuracy: 0.4000
Epoch 2/200
23/23 [==============================] - 13s 565ms/step - loss: 4.3172 - accuracy: 0.7275 - val_loss: 5.0632 - val_accuracy: 0.4375
Epoch 3/200
23/23 [==============================] - 13s 574ms/step - loss: 2.1590 - accuracy: 0.8230 - val_loss: 0.8805 - val_accuracy: 0.8375
Epoch 4/200
23/23 [==============================] - 41s 2s/step - loss: 5.34